# Capstone Project: [testfit.io](https://blog.testfit.io/)
## Wrangle

In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import wrangle as wr
import preprocessing as pr

import warnings
warnings.filterwarnings("ignore")


Bad key "text.kerning_factor" on line 4 in
/usr/local/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
pd.set_option("display.max_columns", None)
plt.rc("figure", figsize=(16, 8))

## Stakeholder Notes
- Work with Commitments
- Activity Descriptions == New Construction, SubRehab, Additions, and Improvements
- Loan amount >= $1M:
    - Mortgage at time of Firm Commitment Issuance, Amendment or Reissuance?
    - Mortgage at Firm Commitment Issuance?
    - Final Mortgage Amount? **Probably this one...**
- Market == Large urban centers; meaning **cities** not just states; not rural one-off projects
- Types of FHA loan is irrelevant

In [ ]:
# commitments
# df  = pd.read_excel("Initi_Endores_Firm Comm_DB_FY06_FY20_Q2.xlsx", sheet_name="Firm Cmtmts, Iss'd and Reiss'd", header=6)
# print(df.shape)
# df.head()
df = pd.read_csv("hud_commitments.csv", index_col=0)
print(df.shape)
df.head()

In [ ]:
# df["Activity Description"].value_counts()

In [ ]:
# df.to_csv("hud_commitments.csv")

In [ ]:
# df.rename(columns={"Activity Group": "activity_group"}, inplace=True)

In [ ]:
# activity_groups filtered for groups of interest for clifton
# df = df[(df.activity_group == "NC/SR") | (df.activity_group == "Imp/Adds")]

In [ ]:
# only including loans greater than or equal to $1M
# df = df[df["Final Mortgage Amount"] >= 1000000]
# df.head()

In [ ]:
df.info()

In [ ]:
# df["Activity Description"].value_counts()

In [ ]:
# df["activity_group"].value_counts()

In [ ]:
df.rename(columns={"Facility Type": "facility_type"}, inplace=True)

In [ ]:
df.facility_type.value_counts()

In [ ]:
df.rename(columns={"FHA Number": "fha_number"}, inplace=True)

In [ ]:
df.groupby(["Project State"]).fha_number.count().sort_values().plot.barh()
plt.show()

In [ ]:
# df[df["Final Mortgage Amount"] >= 1000000].shape

In [ ]:
df.rename(columns={"Final Mortgage Amount": "final_mortgage"}, inplace=True)

In [ ]:
df.groupby(["Project State"]).final_mortgage.sum().sort_values().plot.barh()

In [ ]:
df["Project City"].value_counts().head(20)

In [ ]:
df.groupby(["Project City", "Fiscal Year of Firm Commitment"]).size().sort_values()

In [ ]:
df.groupby(["Project City", "Fiscal Year of Firm Commitment"])["final_mortgage"].sum().sort_values()

In [ ]:
df[df["Project City"] == "Austin"].groupby("Fiscal Year of Firm Commitment")["final_mortgage"].sum()

In [ ]:
df[df["Project City"] == "Minneapolis"].groupby("Fiscal Year of Firm Commitment")["final_mortgage"].sum()

In [ ]:
# csv works and can be read
df = pd.read_csv("hud_commitments.csv", index_col=0)
df.head()

In [ ]:
df[df["Project City"] == "Austin"].groupby("Fiscal Year of Firm Commitment")["Final Mortgage Amount"].sum()

In [ ]:
df[df["Project City"] == "Austin"].groupby("Fiscal Year of Firm Commitment")["Final Mortgage Amount"].count()

In [ ]:
df[df["Project City"] == "Austin"].groupby("Fiscal Year of Firm Commitment")["Final Mortgage Amount"].size()

In [ ]:
df.shape

In [ ]:
# let's see the number of unique values for earch column
# for col in df.columns:
#     unique = len(df[col].unique())
#     if unique > 10:
#         print()
#         print(f'{col} has {unique} values')
#     else:
#         print()
#         print(f'{col} value count:')
#         print(df[col].value_counts())

There are hospitals in this data.

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
# df["Date of Firm Issue"] = pd.to_datetime(df["Date of Firm Issue"])
# df["Date of Firm Issue"].dt.year

---
## Wrangle

In [ ]:
df = wr.wrangle_hud()
print(df.shape)
df.head()

In [ ]:
# looks like the fiscal_year features are the same
# df[df.fiscal_year_of_firm_commitment_activity == df.fiscal_year_of_firm_commitment]

In [ ]:
df.info()

---
## Explore
### What are the characteristics of loan practices in Houston (2009), Seattle (2010), and Dallas (2012)?
- Do these markets follow a similar trajectory in the years leading up the boom?

In [ ]:
# fmt-clipboard function from terminal == black formatting
# df[df.project_city == "Houston"].groupby(["date_of_firm_commitment_activity"])[
#     "final_mortgage_amount"
# ].sum().reset_index().sort_values(by="date_of_firm_commitment_activity")

In [ ]:
# clifton's time machine markets
threshold_cities = ["Houston", "Seattle", "Dallas"]

# loop through 
for city in threshold_cities:
    # subset for city
    date_index_df = df[df.project_city == city].sort_values(
        "date_of_firm_commitment_activity"
    ).set_index("date_of_firm_commitment_activity")

    # sum of the final_mortgage_amount by date_of_firm_commitment_activity
    date_index_df = (
        date_index_df.groupby(["date_of_firm_commitment_activity", "activity_description"])[
            "final_mortgage_amount"
        ]
        .sum()
        .reset_index()
    )
    
    date_index_df.set_index("date_of_firm_commitment_activity", inplace=True)
    
    # plot
    sns.lineplot(data=date_index_df, x=date_index_df.index, y="final_mortgage_amount", hue="activity_description")
    plt.title(f"Final Mortgage Amount Over Time: {city}")
    plt.xlabel("Date of Firm Commitment Activity")
    plt.ylabel("Final Mortgage Amount")
    plt.show()
    
    yearly_avg = date_index_df.resample('Y').mean()
    sns.lineplot(data=yearly_avg, x=yearly_avg.index, y="final_mortgage_amount")
    plt.title(f"Yearly Average Final Mortgage Amount: {city}")
    plt.xlabel("Date of Firm Commitment Activity")
    plt.ylabel("Final Mortgage Amount")
    plt.show()
    
    two_year_avg = date_index_df.resample('2Y').mean()
    sns.lineplot(data=two_year_avg, x=two_year_avg.index, y="final_mortgage_amount")
    plt.title(f"Two-Year Average Final Mortgage Amount: {city}")
    plt.xlabel("Date of Firm Commitment Activity")
    plt.ylabel("Final Mortgage Amount")
    plt.show()

**Takeaway**: What I observe in the charts above is that for the years that Clifton would have liked to enter into a market, the total final mortgage production for that city droped to $10M or less. We need to know from Clifton which markets he know's it's too late to target to derive a more general metric for labeling markets.

### Hypothesis Testing

$H_0$: The mean final mortgage amount is the same across all activity descriptions

$H_a$: The mean final mortgage amount is different depending on the activity description

In [ ]:
df.activity_description.value_counts()

In [ ]:
# 99% confidence level
alpha = .01

refi = df[df.activity_description == "Refinance"].final_mortgage_amount
nc = df[df.activity_description == "New Construction"].final_mortgage_amount
sr = df[df.activity_description == "SubRehab"].final_mortgage_amount
purch = df[df.activity_description == "Purchase"].final_mortgage_amount
add = df[df.activity_description == "Additions"].final_mortgage_amount
imp = df[df.activity_description == "Improvements"].final_mortgage_amount
ol = df[df.activity_description == "Operating Loss"].final_mortgage_amount

stat, pvalue = stats.f_oneway(refi, nc, sr, purch, add, imp, ol)

print(f"statistic = {stat}")
print(f"  p-value = {pvalue}")
print()
if pvalue < alpha:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")

In [ ]:
plt.figure(figsize=(16, 8))

sns.boxplot(data=df, x="activity_description", y="final_mortgage_amount")
plt.xlabel("Activity Description")
plt.ylabel("Final Mortgage Amount")
plt.show()

**Takeaways**: Looks like there are a lot of outliers that are skewing the data. However, we can clearly see that the new construction mortgages have the highest mean among the population.

$H_0$: The mean final mortgage amount is the same across all facility types

$H_a$: The mean final mortgage is different depending on the facility type

---
#### Detour To Standardize `project_city` Names

In [ ]:
pd.DataFrame(df.groupby(["project_city", "fiscal_year_of_firm_commitment"])["final_mortgage_amount"].sum().sort_values(ascending=False).head(50))

In [ ]:
df.project_city.unique()

In [ ]:
# def standardize_city_name(date):
#     return re.sub(r"(\d{2})/(\d{2})/(\d{2})", r"20\3-\1-\2", date)

In [ ]:
# regex = br""

# df["project_city"] = df.project_city.str.replace()

In [ ]:
# (^[a-z]+)(.)([a-z]+)

In [ ]:
# (^[A-Z][a-z]+)(\s?)([A-Z][a-z]+)

In [ ]:
# df.groupby(["project_city", "fiscal_year_of_firm_commitment"])["final_mortgage_amount"].sum()

In [ ]:
# number of unique cities before project_city string transformations
# len(df.project_city.unique())

In [ ]:
# df["project_city"] = df.project_city.str.strip()

In [ ]:
# df.groupby(["project_city", "fiscal_year_of_firm_commitment"])["final_mortgage_amount"].sum()

In [ ]:
# number of unique cities after project_city .strip()
# len(df.project_city.unique())

In [ ]:
# df["project_city"] = df.project_city.str.title()

In [ ]:
df.groupby(["project_city", "fiscal_year_of_firm_commitment"])["final_mortgage_amount"].sum()

In [ ]:
# number of unique cities after project_city .title() case
len(df.project_city.unique())

In [ ]:
df.groupby(["project_city", "fiscal_year_of_firm_commitment"])[
    "final_mortgage_amount"
].sum().sort_values(ascending=False).head(50)

---
### Back on Track with Hypothesis Testing
$H_0$ : The mean final mortgage amount is the same across all facility types

$H_a$ : The mean final mortgage is different depending on the facility type

In [ ]:
df.facility_type.value_counts()

In [ ]:
# 99% confidence level
alpha = .01

apts_coops = df[df.facility_type == "Apts/Coops"].final_mortgage_amount
apts = df[df.facility_type == "Apts"].final_mortgage_amount 
nursing_icf = df[df.facility_type == "Nursing/ICF"].final_mortgage_amount # intermediate care facility
alf = df[df.facility_type == "Asst'd Livg"].final_mortgage_amount # assisted living facility 
coop_apts_mh_alf = df[df.facility_type == "Coop/Apts/Mobile Home Pk/ALF"].final_mortgage_amount
coop = df[df.facility_type == "Coop"].final_mortgage_amount
bc = df[df.facility_type == "Board & Care"].final_mortgage_amount
hosp = df[df.facility_type == "Hospital"].final_mortgage_amount
health_care = df[df.facility_type == "232 Health Care-NH/ICF, ALF or B&C"].final_mortgage_amount
mobile_home = df[df.facility_type == "Mobile Home Pk"].final_mortgage_amount
sro = df[df.facility_type == "SRO"].final_mortgage_amount # single room occupancy 

stat, pvalue = stats.f_oneway(
    apts_coops,
    apts,
    nursing_icf,
    alf,
    coop_apts_mh_alf,
    coop,
    bc,
    hosp,
    health_care,
    mobile_home,
    sro
)


print(f"statistic = {stat}")
print(f"  p-value = {pvalue}")
print()
if pvalue < alpha:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")

In [ ]:
plt.figure(figsize=(16, 8))

sns.boxplot(data=df, x="facility_type", y="final_mortgage_amount")
plt.xlabel("Facility Type")
plt.ylabel("Final Mortgage Amount")
plt.show()

**Takeaway**: I think we should group some of these facilities together. There seems to be a lot of overlap. Based on the visualization above, Hospitals seem to have the highest variance and mean mortgage amount.

## Preprocessing
Considerations:

- Some loans have been reissued. May need to take unique FHA numbers
- Are we using the proper fiscal year feature?
    - may need to use `fiscal_year_of_firm_commitment_activity`
- We will engineer features here by calculating growth rate, mean mortgages over time, and other *benchmark* metrics

In [ ]:
# df = df[(df.activity_group == "NC/SR") | (df.activity_group == "Imp/Adds")]

In [ ]:
# grouping by city, state, and fiscal year or firm commitment
# aggregating final_mortgage_amount 
# sorting by sum and count in descending order
# model_df = df.groupby(["project_city", "project_state", "fiscal_year_of_firm_commitment", "activity_description"])[
#     "final_mortgage_amount"
# ].agg(["count", "sum", "mean", "median"]).sort_values(
#     by=["sum", "count"], ascending=False
# ).reset_index()

In [ ]:
# print(model_df.shape)
# model_df.head(10)

In [ ]:
# model_df.rename(
#     columns={
#         "project_city": "city",
#         "project_state": "state",
#         "fiscal_year_of_firm_commitment": "year",
#         "count": "quantity_of_mortgages",
#         "sum": "total_mortgage_amount",
#         "mean": "average_mortgage_amount",
#         "median": "median_mortgage_amount",
#     },
#     inplace=True,
# )

# model_df.head()

In [ ]:
# df["label"] = np.where(
#     ((df.city == "Houston") & (df.state == "TX") & (df.year == 2009))
#     | ((df.city == "Seattle") & (df.state == "WA") & (df.year == 2010))
#     | ((df.city == "Dallas") & (df.state == "TX")  & (df.year == 2012)),
#     True,
#     False,
# )
# df.head()

In [ ]:
# df[df["label"] == True]

In [ ]:
df = pr.get_model_df()

In [ ]:
print(df.shape)
df.head(10)

In [ ]:
df[df.label == True]

In [ ]:
df.sort_values(by=["quantity_of_mortgages"], ascending=False)

In [ ]:
df[(df.city == "Trenton") & (df.state == "NJ")].sort_values(by=["year"])

---
Standup

Noah Observations perspective:
- Observation of city/state/year for each 
- **Essentially, no missing data for year**
- Check; understood

Other notes:
- OHE activity description
- Counting number of refinances for city/state/year
- Ratio of refi to nc
- Bool for for is_refi:
    - mean for ratio of refi
    
Maggie:
- Labels
- Feature city interest based on regulation
    - NYC too heavily regulated
    - More blue cities are more heavily regulated
    - Weight of regulation?
        - Interest index due to regulation
        - If the index is high == high barrier, low == low barrier
        - Defining by an interview

In [ ]:
df

In [ ]:
pd.DataFrame(df.groupby(["city", "state"]).size().reset_index())

In [ ]:
city_state = pd.DataFrame(df.groupby(["city", "state"]).size() == 14)

In [ ]:
city_state = city_state.reset_index()
city_state

In [ ]:
city_state.rename(columns={0: "observations"}, inplace=True)

In [ ]:
# these are the city + state combinations that have 14 consecutive year observations
city_state[city_state.observations == True]

In [ ]:
# df["observations"] = np.where(df.groupby(["city", "state"]).size() == 14, True, False)

In [ ]:
diff_df = pd.DataFrame(df.year.groupby([df.city, df.state]).diff().eq(14))
diff_df

In [ ]:
print(diff_df[diff_df.year == True].shape)
diff_df[diff_df.year == True]

In [ ]:
df = wr.wrangle_hud()

In [ ]:
df.head()

In [ ]:
# nc = df[df.activity_description == "New Construction"]
# nc.head()

In [ ]:
# sns.lineplot(data=nc, x="date_of_firm_commitment_activity", y="final_mortgage_amount")
# plt.show()

In [ ]:
# sns.lineplot(data=nc, x="date_of_firm_commitment_activity", y="final_mortgage_amount")

In [ ]:
city_state_all_act = pd.DataFrame(
    df.groupby(["project_city", "project_state", "fiscal_year_of_firm_commitment_activity", "activity_description"])["final_mortgage_amount"]
    .agg(["count", "sum"])
    .reset_index()
    .sort_values(by=["count", "sum"], ascending=False)
)
city_state_all_act

**Preprocessing**: 

In [ ]:
sns.lineplot(
    data=city_state_all_act,
    x="fiscal_year_of_firm_commitment_activity",
    y="sum",
    hue="activity_description",
    ci=False
)


**Takeaway**: NC + SR growing steadily since 2014. Overall, NC is where most of the mortgage loans are being allocated. If we can find cities that are spending a lot on NC, we can ID those as markets to enter. Feature: calculating the $ of NC and quantity for all cities and years, then we can classsify the 

In [ ]:
atl = nc_cities[(nc_cities.project_city == "Atlanta") & (nc_cities.project_state == "GA")]
atl

In [ ]:
sns.lineplot(data=atl, x="fiscal_year_of_firm_commitment_activity", y="sum")

In [ ]:
nc_cities = pd.DataFrame(
    nc.groupby(["project_city", "project_state", "date_of_firm_commitment_activity"])["final_mortgage_amount"]
    .agg(["count", "sum"])
    .reset_index()
    .sort_values(by=["count", "sum"], ascending=False)
)
nc_cities

In [ ]:
sns.lineplot(data=nc_cities, x="date_of_firm_commitment_activity", y="sum")

---
### Preprocessing
Tasks:
- Add New Construction (NC) count and sum to preprocessing df
- Add ratios of NC to the total number of mortgages and sum of mortgages

In [22]:
df = wr.wrangle_hud()
print(df.shape)
df.head()

(20917, 29)


,fha_number,project_name,project_city,project_state,basic_fha_risk_share_or_other,program_category,activity_description,activity_group,facility_type,program_designation,firm_commitment_activity,lender_at_firm_commitment_activity,mortgage_at_time_of_firm_commitment_issuance_amendment_or_reissuance,unit_or_bed_count,date_of_firm_commitment_activity,fiscal_year_of_firm_commitment_activity,mortgage_at_firm_commitment_issuance,date_of_firm_issue,fiscal_year_of_firm_commitment,map_or_tap,lihtc_designation,tax_exempt_bond_designation,home_designation,cdbg_designation,section_202_refi_designation,irp_decoupling_designation,hope_vi_designation,current_status,final_mortgage_amount
0,11345004,Western Heights Apartments,Olney,TX,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/236,Firm Issued,GREYSTONE SERVICING COMPANY LLC,277600,48,2005-10-01,2006,277600,2005-10-01,2006,TAP,False,False,False,False,False,False,False,Firm Comit then Owner Withdrew,271700
1,11345005,Woodlands Village Apts,Breckenridge,TX,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/236,Firm Issued,GREYSTONE SERVICING COMPANY LLC,173000,88,2005-10-01,2006,173000,2005-10-01,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,168300
2,4235583,Harbor Court Apartments,Rocky River,OH,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/221d4,Firm Issued,GREYSTONE SERVICING COMPANY LLC,3886800,120,2005-10-03,2006,3886800,2005-10-03,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,3886800
3,3297260,Compton Towne Apartments,Wilmington,DE,Risk Sharing,QPE Risk Sharing,Refinance,Refi/ Pchse,Coop/Apts/Mobile Home Pk/ALF,QPE Risk Shg Existing,Firm Issued,FEDERAL NATIONAL MORTGAGE,4000000,76,2005-10-03,2006,4000000,2005-10-03,2006,TAP,True,False,False,False,False,False,False,Finally Endorsed,4000000
4,7135779,Evergreen Terrace I,Joliet,IL,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/221d4,Firm Issued,REILLY MORTGAGE GRP,5129000,241,2005-10-04,2006,5129000,2005-10-04,2006,TAP,False,False,False,False,False,False,False,Firm Expired,5247700


In [23]:
def preprocessing_for_modeling(df):
    """function to manipulate df into df usable for modeling"""
    # city mask identifies 35 cities with most hud loans
    city_mask = df.project_city.value_counts().nlargest(35)
    # apply city mask to shrink the df
    def in_city_mask(x):
        return x in city_mask
    df = df[df.project_city.apply(in_city_mask)]
    #create a df for modelling that groups-by year and city aggregating mortgage amount by count, median, mean, and sum
    df_for_model = (df.groupby(['fiscal_year_of_firm_commitment_activity', 'project_city'])
         .final_mortgage_amount.agg(['count', 'median', 'mean', 'sum']).reset_index())
    df_for_model = df_for_model.rename(columns={'fiscal_year_of_firm_commitment_activity': 'year', 'project_city': 'city'})
    return df_for_model

In [24]:
df = preprocessing_for_modeling(df)
df

,year,city,count,median,mean,sum
0,2006,Atlanta,6,4457100.0,5.531133e+06,33186800
1,2006,Austin,1,14083843.0,1.408384e+07,14083843
2,2006,Baltimore,6,4570450.0,5.993167e+06,35959000
3,2006,Boston,7,3941300.0,7.059614e+06,49417300
4,2006,Bronx,7,1155300.0,3.298414e+06,23088900
...,...,...,...,...,...,...
505,2020,Portland,4,11486550.0,1.150535e+07,46021400
506,2020,Richmond,1,23534500.0,2.353450e+07,23534500
507,2020,San Antonio,4,17698200.0,1.791435e+07,71657400
508,2020,St. Louis,3,6316200.0,7.585433e+06,22756300


In [25]:
# group = raw_df.groupby(["project_city", "fiscal_year_of_firm_commitment_activity"])
# pd.DataFrame(
#     group.apply(
#         lambda x: x[x["activity_description"] == "New Construction"][
#             "final_mortgage_amount"
#         ].sum()
#     )
# ).reset_index()


In [26]:
df.city.value_counts()

New York        15
Los Angeles     15
St. Louis       15
Chicago         15
Columbia        15
Columbus        15
Minneapolis     15
Baltimore       15
Houston         15
Dallas          15
Cincinnati      15
Phoenix         15
Atlanta         15
Orlando         15
Austin          15
Indianapolis    15
Portland        15
Memphis         15
Boston          15
Washington      15
Detroit         15
Cleveland       15
Greenville      14
Milwaukee       14
San Antonio     14
Fort Worth      14
Richmond        14
Bronx           14
Brooklyn        14
Denver          14
Madison         14
Jacksonville    14
Charlotte       14
Knoxville       13
Toledo          13
Name: city, dtype: int64

In [27]:
# What is the time range?

df.year.min(), df.year.max() 

(2006, 2020)

In [28]:
# mean rate of change

df["mean_var"] = df.sort_values(["city", "year"])["mean"].pct_change()

In [29]:
df = df.sort_values(["city", "year"])

In [30]:
df

,year,city,count,median,mean,sum,mean_var
0,2006,Atlanta,6,4457100.0,5.531133e+06,33186800,NaN
34,2007,Atlanta,9,2990200.0,3.233978e+06,29105800,-0.415314
67,2008,Atlanta,2,3367500.0,3.367500e+06,6735000,0.041287
99,2009,Atlanta,9,4400000.0,5.552544e+06,49972900,0.648862
131,2010,Atlanta,11,5196100.0,6.253490e+06,68788385,0.126239
...,...,...,...,...,...,...,...
373,2016,Washington,8,27352400.0,3.910451e+07,312836100,0.445886
407,2017,Washington,8,11693000.0,1.719466e+07,137557300,-0.560290
442,2018,Washington,7,25450000.0,2.417843e+07,169249000,0.406159
476,2019,Washington,5,9300000.0,1.234469e+07,61723450,-0.489434


In [31]:
# count rate of change

df["count_var"] = df.sort_values(["city", "year"])["count"].pct_change()
df

,year,city,count,median,mean,sum,mean_var,count_var
0,2006,Atlanta,6,4457100.0,5.531133e+06,33186800,NaN,NaN
34,2007,Atlanta,9,2990200.0,3.233978e+06,29105800,-0.415314,0.500000
67,2008,Atlanta,2,3367500.0,3.367500e+06,6735000,0.041287,-0.777778
99,2009,Atlanta,9,4400000.0,5.552544e+06,49972900,0.648862,3.500000
131,2010,Atlanta,11,5196100.0,6.253490e+06,68788385,0.126239,0.222222
...,...,...,...,...,...,...,...,...
373,2016,Washington,8,27352400.0,3.910451e+07,312836100,0.445886,0.000000
407,2017,Washington,8,11693000.0,1.719466e+07,137557300,-0.560290,0.000000
442,2018,Washington,7,25450000.0,2.417843e+07,169249000,0.406159,-0.125000
476,2019,Washington,5,9300000.0,1.234469e+07,61723450,-0.489434,-0.285714


In [32]:
# evolution index
# EI = (1 + Company Growth %) / (1 + Market Growth %) X 100

# How can I calculate the market growth for each year?

df["market_growth"] = df.groupby("year").mean_var.transform("mean")

In [33]:
df

,year,city,count,median,mean,sum,mean_var,count_var,market_growth
0,2006,Atlanta,6,4457100.0,5.531133e+06,33186800,NaN,NaN,-0.539085
34,2007,Atlanta,9,2990200.0,3.233978e+06,29105800,-0.415314,0.500000,0.548512
67,2008,Atlanta,2,3367500.0,3.367500e+06,6735000,0.041287,-0.777778,0.594319
99,2009,Atlanta,9,4400000.0,5.552544e+06,49972900,0.648862,3.500000,1.395160
131,2010,Atlanta,11,5196100.0,6.253490e+06,68788385,0.126239,0.222222,1.684237
...,...,...,...,...,...,...,...,...,...
373,2016,Washington,8,27352400.0,3.910451e+07,312836100,0.445886,0.000000,0.490671
407,2017,Washington,8,11693000.0,1.719466e+07,137557300,-0.560290,0.000000,0.403235
442,2018,Washington,7,25450000.0,2.417843e+07,169249000,0.406159,-0.125000,0.348374
476,2019,Washington,5,9300000.0,1.234469e+07,61723450,-0.489434,-0.285714,0.201454


In [34]:
df["ei"] = (1+ df.mean_var) / (1+ df.market_growth) * 100

In [35]:
model_df = df
model_df

,year,city,count,median,mean,sum,mean_var,count_var,market_growth,ei
0,2006,Atlanta,6,4457100.0,5.531133e+06,33186800,NaN,NaN,-0.539085,NaN
34,2007,Atlanta,9,2990200.0,3.233978e+06,29105800,-0.415314,0.500000,0.548512,37.757946
67,2008,Atlanta,2,3367500.0,3.367500e+06,6735000,0.041287,-0.777778,0.594319,65.312351
99,2009,Atlanta,9,4400000.0,5.552544e+06,49972900,0.648862,3.500000,1.395160,68.841431
131,2010,Atlanta,11,5196100.0,6.253490e+06,68788385,0.126239,0.222222,1.684237,41.957492
...,...,...,...,...,...,...,...,...,...,...
373,2016,Washington,8,27352400.0,3.910451e+07,312836100,0.445886,0.000000,0.490671,96.995684
407,2017,Washington,8,11693000.0,1.719466e+07,137557300,-0.560290,0.000000,0.403235,31.335472
442,2018,Washington,7,25450000.0,2.417843e+07,169249000,0.406159,-0.125000,0.348374,104.285561
476,2019,Washington,5,9300000.0,1.234469e+07,61723450,-0.489434,-0.285714,0.201454,42.495695


In [36]:
# example from article
# def test_sum(series):
#     return reduce(lambda x, y: x + y, series)

In [37]:
# write a lambda square function
# square = lambda x: x ** 2

In [38]:
# square(9)

In [39]:
# raw_df.groupby(
#     ["fiscal_year_of_firm_commitment_activity", "project_city"]
# ).final_mortgage_amount.agg(["sum"])

In [42]:
df = wr.wrangle_hud()
print(df.shape)
df.head()

(20917, 29)


,fha_number,project_name,project_city,project_state,basic_fha_risk_share_or_other,program_category,activity_description,activity_group,facility_type,program_designation,firm_commitment_activity,lender_at_firm_commitment_activity,mortgage_at_time_of_firm_commitment_issuance_amendment_or_reissuance,unit_or_bed_count,date_of_firm_commitment_activity,fiscal_year_of_firm_commitment_activity,mortgage_at_firm_commitment_issuance,date_of_firm_issue,fiscal_year_of_firm_commitment,map_or_tap,lihtc_designation,tax_exempt_bond_designation,home_designation,cdbg_designation,section_202_refi_designation,irp_decoupling_designation,hope_vi_designation,current_status,final_mortgage_amount
0,11345004,Western Heights Apartments,Olney,TX,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/236,Firm Issued,GREYSTONE SERVICING COMPANY LLC,277600,48,2005-10-01,2006,277600,2005-10-01,2006,TAP,False,False,False,False,False,False,False,Firm Comit then Owner Withdrew,271700
1,11345005,Woodlands Village Apts,Breckenridge,TX,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/236,Firm Issued,GREYSTONE SERVICING COMPANY LLC,173000,88,2005-10-01,2006,173000,2005-10-01,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,168300
2,4235583,Harbor Court Apartments,Rocky River,OH,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/221d4,Firm Issued,GREYSTONE SERVICING COMPANY LLC,3886800,120,2005-10-03,2006,3886800,2005-10-03,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,3886800
3,3297260,Compton Towne Apartments,Wilmington,DE,Risk Sharing,QPE Risk Sharing,Refinance,Refi/ Pchse,Coop/Apts/Mobile Home Pk/ALF,QPE Risk Shg Existing,Firm Issued,FEDERAL NATIONAL MORTGAGE,4000000,76,2005-10-03,2006,4000000,2005-10-03,2006,TAP,True,False,False,False,False,False,False,Finally Endorsed,4000000
4,7135779,Evergreen Terrace I,Joliet,IL,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/221d4,Firm Issued,REILLY MORTGAGE GRP,5129000,241,2005-10-04,2006,5129000,2005-10-04,2006,TAP,False,False,False,False,False,False,False,Firm Expired,5247700


In [43]:
# number of new construction projects
len(df[df.activity_description == "New Construction"])

2479

In [45]:
# ratio of new construction projects to total number of projects
len(df[df["activity_description"] == "New Construction"]) / len(df)

0.11851603958502653

In [46]:
# total number of mortgage dollars for new construction projects
df[df["activity_description"] == "New Construction"].final_mortgage_amount.sum()

50115653952

In [47]:
# ratio of new construction final_mortgage_amount to total final_mortgage_amount of all projects
df[
    df["activity_description"] == "New Construction"
].final_mortgage_amount.sum() / df.final_mortgage_amount.sum()

0.22336668010301566

In [48]:
# def new_construction(x):
    
#     nc_final_mortgage_amount = x.sum()
    
#     return nc_final_mortgage_amount if m > 0.5 else 0 if m < 0.5 else np.nan

In [49]:
# group = raw_df.groupby(["project_city", "fiscal_year_of_firm_commitment_activity"])
# pd.DataFrame(
#     group.apply(
#         lambda x: x[x["activity_description"] == "New Construction"][
#             "final_mortgage_amount"
#         ].sum()
#     )
# ).reset_index()

In [50]:
# df.groupby(["fiscal_year_of_firm_commitment_activity", "project_city"])[
#     "final_mortgage_amount"
# ].agg(["count", "sum", "mean", "median"])

In [51]:
# def new_construction_mortgages(x):
#     return raw_df[raw_df["activity_description"] == "New Construction"].final_mortgage_amount.sum()

In [52]:
# df.groupby(["fiscal_year_of_firm_commitment_activity", "project_city"])[
#     "final_mortgage_amount"
# ].agg(["count", "sum", "mean", "median", "new_construction_mortgages"])

In [53]:
nc_df = df[df.activity_description == "New Construction"]
nc_df

,fha_number,project_name,project_city,project_state,basic_fha_risk_share_or_other,program_category,activity_description,activity_group,facility_type,program_designation,firm_commitment_activity,lender_at_firm_commitment_activity,mortgage_at_time_of_firm_commitment_issuance_amendment_or_reissuance,unit_or_bed_count,date_of_firm_commitment_activity,fiscal_year_of_firm_commitment_activity,mortgage_at_firm_commitment_issuance,date_of_firm_issue,fiscal_year_of_firm_commitment,map_or_tap,lihtc_designation,tax_exempt_bond_designation,home_designation,cdbg_designation,section_202_refi_designation,irp_decoupling_designation,hope_vi_designation,current_status,final_mortgage_amount
9,5235624,Port Capital Village,Elkridge,MD,Basic FHA,FHA NC/SR Apts,New Construction,NC/SR,Apts,221d4,Firm Issued,AGM FINANCIAL SERVICES,5410000,84,2005-10-06,2006,5410000,2005-10-06,2006,MAP,False,True,True,False,False,False,False,Finally Endorsed,5410000
29,6135628,Veranda at the Villages at Carver,Atlanta,GA,Basic FHA,FHA NC/SR Apts,New Construction,NC/SR,Apts,221d4,Firm Issued,AMERICAN PROPERTY FINCG,3988000,90,2005-10-14,2006,3988000,2005-10-14,2006,MAP,True,False,False,False,False,False,False,Finally Endorsed,3988000
41,9235668,St. Anthony Mills Apartments,Minneapolis,MN,Basic FHA,FHA NC/SR Apts,New Construction,NC/SR,Apts,221d4,Firm Issued,GLASER FINANCIAL GRP,6140000,93,2005-10-17,2006,6140000,2005-10-17,2006,TAP,True,True,True,False,False,False,False,Finally Endorsed,6120000
50,6135631,Peabody Apartments,Columbus,GA,Basic FHA,FHA NC/SR Apts,New Construction,NC/SR,Apts,221d4,Firm Issued,"CAPMARK FINANCE, INC",3335000,184,2005-10-19,2006,3335000,2005-10-19,2006,MAP,True,False,False,False,False,False,True,Finally Endorsed,3335000
53,12335422,The Retreat at West Point Apartments,Surprise,AZ,Basic FHA,FHA NC/SR Apts,New Construction,NC/SR,Apts,221d4,Firm Issued,PARAGON MORTGAGE,12438700,142,2005-10-19,2006,12438700,2005-10-19,2006,MAP,False,False,False,False,False,False,False,Finally Endorsed,12438700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20883,8735318,KINGSTON PIKE VILLAGE,Farragut,TN,Basic FHA,FHA NC/SR Apts,New Construction,NC/SR,Apts,221d4,Firm Issued,NEWMARK KNIGHT FRANK,33860600,228,2020-03-24,2020,33860600,2020-03-24,2020,MAP,False,False,False,False,False,False,False,Firm Issued,33860600
20888,2398425,Mattapan Station 9%,Boston,MA,Risk Sharing,HFA Risk Sharing,New Construction,NC/SR,Coop/Apts/Mobile Home Pk/ALF,HFA Risk Shg NC/SR,Firm Issued,MASSACHUSETTS HFA,2140000,21,2020-03-24,2020,2140000,2020-03-24,2020,TAP,True,False,False,False,False,False,False,Firm Amended,2140000
20890,2398432,Whittier Phase 2,Boston,MA,Risk Sharing,HFA Risk Sharing,New Construction,NC/SR,Coop/Apts/Mobile Home Pk/ALF,HFA Risk Shg NC/SR,Firm Issued,MASSACHUSETTS HFA,15731824,52,2020-03-25,2020,15731824,2020-03-25,2020,TAP,True,True,False,False,False,False,False,Firm Amended,15731824
20905,8435405,Metro North Crossing Apartments,Kansas City,MO,Basic FHA,FHA NC/SR Apts,New Construction,NC/SR,Apts,221d4,Firm Issued,GERSHMAN INVESTMENT,50764200,249,2020-03-27,2020,50764200,2020-03-27,2020,MAP,False,False,False,False,False,False,False,Firm Issued,50764200


In [54]:
model_nc_df = preprocessing_for_modeling(nc_df)
model_nc_df

,year,city,count,median,mean,sum
0,2006,Atlanta,2,4590550,4590550.0,9181100
1,2006,Chicago,1,1136900,1136900.0,1136900
2,2006,Columbia,1,17214400,17214400.0,17214400
3,2006,Columbus,1,3335000,3335000.0,3335000
4,2006,Corpus Christi,1,3379100,3379100.0,3379100
...,...,...,...,...,...,...
297,2020,Portland,1,12073200,12073200.0,12073200
298,2020,Raleigh,1,33269100,33269100.0,33269100
299,2020,Salt Lake City,1,45283800,45283800.0,45283800
300,2020,San Antonio,2,27711800,27711800.0,55423600


In [55]:
model_df

,year,city,count,median,mean,sum,mean_var,count_var,market_growth,ei
0,2006,Atlanta,6,4457100.0,5.531133e+06,33186800,NaN,NaN,-0.539085,NaN
34,2007,Atlanta,9,2990200.0,3.233978e+06,29105800,-0.415314,0.500000,0.548512,37.757946
67,2008,Atlanta,2,3367500.0,3.367500e+06,6735000,0.041287,-0.777778,0.594319,65.312351
99,2009,Atlanta,9,4400000.0,5.552544e+06,49972900,0.648862,3.500000,1.395160,68.841431
131,2010,Atlanta,11,5196100.0,6.253490e+06,68788385,0.126239,0.222222,1.684237,41.957492
...,...,...,...,...,...,...,...,...,...,...
373,2016,Washington,8,27352400.0,3.910451e+07,312836100,0.445886,0.000000,0.490671,96.995684
407,2017,Washington,8,11693000.0,1.719466e+07,137557300,-0.560290,0.000000,0.403235,31.335472
442,2018,Washington,7,25450000.0,2.417843e+07,169249000,0.406159,-0.125000,0.348374,104.285561
476,2019,Washington,5,9300000.0,1.234469e+07,61723450,-0.489434,-0.285714,0.201454,42.495695


In [ ]:
pd.merge()